## Предобработка

Функции предобработки для одного документа и для всего корпуса

In [4]:
import os
from tqdm.notebook import tqdm
import warnings

import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords
import re
from pymorphy3 import MorphAnalyzer

In [ ]:
morph = MorphAnalyzer()
nltk.download('stopwords')  # Будут предупреждения, если пакет уже загружен - это нормально
stop_words = set(stopwords.words('russian'))


def preprocess_text(text: str) -> list[str]:
    """
    Предобработка текста для сентимент-анализа: удаление стоп-слов и лишних символов,
    лемматизация, токенизация
    :param text: Текст, который надо обработать
    :return: Список обработанных токенов
    """

    text = text.lower()
    text = re.sub(r'[^а-яёЁ?!-+0-9\s]', '', text)
    text = text.replace('!', ' ! ').replace('!?', ' !? ')
    text = re.sub(r'[()\[\]{}]', '', text)

    tokens = text.split()
    tokens = [morph.parse(token)[0].normal_form for token in tokens if token not in stop_words]
    return tokens


def preprocess(dataset: pd.DataFrame) -> None:
    """
    Создание дополнительного признака в датасете - результат применения preprocess_text к тексту.
    :param dataset: Датасет, который надо обработать
    :return: None
    """

    tqdm.pandas(desc="Обработка датасета")

    dataset["prep_text"] = dataset["text"].progress_apply(preprocess_text)

    return None

Применение функции предобработки к датасетам и их сохранение в зависимости от того, было ли это сделано ранее.

In [6]:
data_dir = "../data"

prep_train_path = os.path.join(data_dir, 'prep_train.csv')
prep_test_path = os.path.join(data_dir, 'prep_test.csv')

if not os.path.exists(prep_train_path):
    print("Обработка тренировочного датасета")
    preprocess(train)
    train.to_csv(prep_train_path, index=False)  # Сохранение обработанных данных
    print(f"Обработанные тренировочные данные сохранены")

if not os.path.exists(prep_test_path):
    print("Обработка тестового датасета")
    preprocess(test)
    test.to_csv(prep_test_path, index=False)  # Сохранение обработанных данных
    print(f"Обработанные тестовые данные сохранены")

prep_train = pd.read_csv(os.path.join(data_dir, 'prep_train.csv'))
prep_test = pd.read_csv(os.path.join(data_dir, 'prep_test.csv'))